In [1]:
import numpy as np
import scipy as sc
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from pandas import DataFrame, Series
from datetime import date, datetime, timedelta
import radar
import random

In [2]:
df = pd.read_csv('/Users/idanattias/Downloads/workshop/requests.csv')

/Users/idanattias/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#leave only informative columns
df_subset = df.filter(items = ['TimeStamp', 'RoleInst', 'Continent', 'OpName', 'Country', 'Host', 'Response', 'ReqDuration'])
df_subset = df_subset.replace(np.nan, 'Null', regex=True)

#convert columns to proper type
df_subset['TimeStamp'] = pd.to_datetime(df_subset['TimeStamp'])
categorical_columns = list(set(df_subset.columns.values) - set(['TimeStamp', 'ReqDuration']))
for col in categorical_columns:
    df_subset[col] = df_subset[col].astype('category')

#subset df to contain only full data (continuous data flow stopped on 2015-10-07 at 18:00)
df_subset = df_subset[df_subset['TimeStamp'] < pd.to_datetime('2015-10-07 18:00')]
df_subset = df_subset.sort_values(by = 'TimeStamp')

#print ('\ninfo:')
#print df_subset.info()

In [4]:
#remove outliers: keep only the ones that are within +3 to -3 standard deviations in the column 'ReqDuration'
df_subset=df_subset[np.abs(df_subset.ReqDuration-df_subset.ReqDuration.mean())<=(3*df_subset.ReqDuration.std())]

#df_subset
#df_subset.plot(x='TimeStamp',y='ReqDuration')
#plt.show()

In [5]:
#these variables store essential information about the data
req_dur_sum=df_subset['ReqDuration'].sum()
req_dur_per_day=[0 for i in range(7)]
req_dur_per_hour=[[0 for i in range(24)] for i in range(7)]
num_of_events_per_hour=[[0 for i in range(24)] for i in range(7)]
num_of_events_per_day=[0 for i in range(7)]
mean_per_hour=[[0 for i in range(24)] for i in range(7)] #average, not mean!
new_mean_per_hour= [[0 for i in range(24)] for i in range(7)]
std_per_hour= [[0 for i in range(24)] for i in range(7)]
mean_per_day= [0 for i in range(7)]
fraction_of_hour_per_day= [[0 for i in range(24)] for i in range(7)]
fraction_of_typical_hour= [0 for i in range(24)]

list_for_calc_std=[]

day=df_subset.iloc[0,0].day
hour=df_subset.iloc[0,0].hour

#update arrays above
for index, event in df_subset.iterrows():
    if day < event['TimeStamp'].day:
        #update std for each hour
        std_per_hour[day-1][hour]=np.std(list_for_calc_std)
        day+=1
        hour=0
        list_for_calc_std=[]
    
    if hour < event['TimeStamp'].hour:
        #update std for each hour
        std_per_hour[day-1][hour]=np.std(list_for_calc_std)
        hour+=1
        list_for_calc_std=[]
    
    list_for_calc_std.append(event['ReqDuration'])
    
    req_dur_per_day[day-1]+= event['ReqDuration']
    req_dur_per_hour[day-1][hour] += event['ReqDuration']
    num_of_events_per_hour[day-1][hour]+=1


In [6]:
## updates of information about the data ##

#update mean for each hour
for i in range (7):
    for j in range(24):
        if (num_of_events_per_hour[i][j]==0):
            mean_per_hour[i][j]=0
        else:
            mean_per_hour[i][j]= req_dur_per_hour[i][j]/num_of_events_per_hour[i][j]
            
#update number of events per day
temp=0
for i in range (7):
    for j in range(24):
        temp+= num_of_events_per_hour[i][j]
        if(j==23):
            num_of_events_per_day[i]=temp
            temp=0



#update mean per day
for i in range(7):
    mean_per_day[i] = req_dur_per_day[i]/num_of_events_per_day[i]
    

#update fraction_of_hour_per_day
for i in range (7):
    for j in range(24):
        fraction_of_hour_per_day[i][j] = mean_per_hour[i][j]/mean_per_day[i]

# modify to dataframe
headers=[i+1 for i in range(24)]
df_fraction_of_hour_per_day= DataFrame(fraction_of_hour_per_day, columns=headers)


sum_of_columns=df_fraction_of_hour_per_day.sum(axis=0)
sum_of_fraction_of_typical_hour =sum_of_columns.tolist()


for i in range (24):
    if (j>=18):
        fraction_of_typical_hour[i]= sum_of_fraction_of_typical_hour[i]/6
    else:
        fraction_of_typical_hour[i]= sum_of_fraction_of_typical_hour[i]/7

for i in range(7):
    for j in range(24):
        new_mean_per_hour[i][j]= mean_per_day[i]*fraction_of_typical_hour[j]


In [7]:
## creating the synthetic data starts here. first phase: create clean data ##

artificial_df = pd.DataFrame(columns=['TimeStamp','ReqDuration'])
std= 70

for i in range (7):
    for j in range(24):
        
        #edge case
        if(j==23): 
            start = datetime(year=2015, month=10, day=i+1,hour=j,minute=0,second=0)
            stop = datetime(year=2015, month=10, day=i+2,hour=0,minute=0,second=0)
       
        else:
            start = datetime(year=2015, month=10, day=i+1,hour=j,minute=0,second=0)
            stop = datetime(year=2015, month=10, day=i+1,hour=j+1,minute=0,second=0)
        
        temp_timestamps=[]
       
        #create random time stamps
        #4144= num of events per hour on average
        for event in xrange(4144): 
            temp_timestamps.append(radar.random_datetime(start=start, stop=stop))
        
        #create points with normal distribution for each hour
       
        #if(i==6 and j>=17):   #edge case: this hour has no data
        #    std=1                    
        #else: 
        #   std=std_per_hour[i][j]
       

        temp_points= list(np.random.normal(new_mean_per_hour[i][j],std,4144))
        
        #create list of tuples: (timestamp,value)
        zipped=zip(temp_timestamps,temp_points)
        
        #modify it to list of lists
        new_data_for_hour = [list(elem) for elem in zipped]
       
        temp_df= pd.DataFrame(new_data_for_hour, columns=['TimeStamp','ReqDuration'])   
        artificial_df= artificial_df.append(temp_df, ignore_index=True) #this function returns a new object
    
artificial_df= artificial_df.sort_values(by = 'TimeStamp')
artificial_df.index = range(696192) #change row indices: to ascending order

#### till here, we have a clean generic synthetic data ####

#artificial_df.plot(x='TimeStamp',y='ReqDuration')
#plt.show()

In [8]:
## create feature coulmns and add it to artificial_df ##
feature_uniform = np.random.randint(1,5,696192) #discrete uniform distribution: 4 categories (1-4)
feature_binomial1= np.random.binomial(4,0.5,696192) #binomial distribution : 5 categories (0-4)
feature_binomial2= np.random.binomial(3, 0.3,696192) #binomial distribution : 4 categories (0-3)
feature_poisson= np.random.poisson(1,696192) #poisson distribution : approximately 5 categories (0-4)

#add coulmns to the end of the dataframe
artificial_df['uniform_feature']=feature_uniform
artificial_df['binomial1_feature']=feature_binomial1
artificial_df['binomial2_feature']=feature_binomial2
artificial_df['poisson_feature']=feature_poisson


##rearrange coulmns : ReqDuration supposed to be at the end of the frame
cols = list(artificial_df)
# move the ReqDuration column to the end of the list using index, pop and insert
cols.insert(len(cols), cols.pop(cols.index('ReqDuration')))
artificial_df = artificial_df.ix[:, cols] 

#### after this phase we have a clean synthetic with the columns: time stamp, 4 features, request duration ####

In [9]:
## add outliers in random places by choosing randomly "problemtic" features ##

num_of_outliers=20 ##trying to add this number of outliers. apperantly will be less outliers:
                    ##just if the events in the time stamps will contain the tuples
feature_names= ['uniform_feature','binomial1_feature','binomial2_feature','poisson_feature']
recorded_outliers=[] ## will contain the "causes" for the outliers
timestamps_with_outlier=[]

for outlier in range (num_of_outliers):
    #parameters
    start_index_of_outlier= np.random.randint(0,696192,1)[0] #random start point of outlier
    duration_of_outlier= np.random.randint(150,650,1)[0] #duration of anomaly, runs over num of indices
                                                            #approximately 2-10 minutes
    strength_of_outlier = np.random.randint(15,30,1)[0] # for differnt kind of outliers
                                                        #our distance in std from the other values
    num_of_featurs_in_anomaly= np.random.randint(1,4,1)[0] #allow 1-3 features optionally in anomaly
    rand_features = random.sample(feature_names,num_of_featurs_in_anomaly) #list of random features
    
    #generate (uniformly) random values for each feature. e.g if country was chosen- choose which country
    rand_values_of_features= []
    for k in range (len(rand_features)):
        if (rand_features[k]=='uniform_feature'):
            rand_values_of_features.append(np.random.randint(1,5,1)[0])  
        elif (rand_features[k]=='binomial1_feature'):
            rand_values_of_features.append(np.random.randint(0,5,1)[0])
        elif (rand_features[k]=='binomial2_feature'):
            rand_values_of_features.append(np.random.randint(0,4,1)[0])            
        elif (rand_features[k]== 'poisson_feature'):
            rand_values_of_features.append(np.random.randint(0,5,1)[0]) 
    
    
    #insert the outlier: run over the events, if tuple was found- update request duration
    how_many_events_were_affected=0
    cnt=start_index_of_outlier
    while (cnt<start_index_of_outlier + duration_of_outlier):
        event_contains_feature=True
      
        #check if event contains the chosen tuple
        for j in range(num_of_featurs_in_anomaly):
            if(artificial_df.loc[cnt,rand_features[j]]!= rand_values_of_features[j]):
                event_contains_feature=False
                break
        
        #update new request duration for this event
        if(event_contains_feature):
            timestamps_with_outlier.append(artificial_df.loc[cnt,'TimeStamp'])
            new_req_duration= artificial_df.loc[cnt,'ReqDuration']+ std*strength_of_outlier
            artificial_df.set_value(cnt, 'ReqDuration', new_req_duration)
            how_many_events_were_affected+=1 
            
        cnt+=1
    
    #if the outlier has an impact: update recorded_outliers
    if (how_many_events_were_affected != 0):
        start_time=artificial_df.loc[start_index_of_outlier,'TimeStamp']
        end_time=artificial_df.loc[start_index_of_outlier + duration_of_outlier ,'TimeStamp']
        feature_values=[None for i in range(4)]
        
        for index,feature in enumerate(rand_features):
            if (feature =='uniform_feature'):
                feature_values[0]=rand_values_of_features[index]
            elif (feature == 'binomial1_feature'):
                feature_values[1]=rand_values_of_features[index]
            elif (feature == 'binomial2_feature'):
                 feature_values[2]=rand_values_of_features[index]
            elif (feature == 'poisson_feature'):
                feature_values[3]=rand_values_of_features[index]
                
        temp_list=[start_time,end_time,strength_of_outlier]
        
        for val in feature_values:
            temp_list.append(val)
        
        #each temp_list is anomaly. add it to the list of anomalies
        recorded_outliers.append(temp_list)

        
#update the artificial anomlies added by us
headers=['start time','end time','strength','uniform feature','binomial1 feature','binomial2 feature', 'poisson feature']        
df_recorded_outliers = DataFrame(recorded_outliers, columns=headers)
df_recorded_outliers = df_recorded_outliers.sort_values(by = 'start time') #sort timestamps
df_recorded_outliers.index = range(len(recorded_outliers)) #change row indices: to ascending order


#### after this phase we have a synthetic data with artficial outliers made by us ####
#### the outliers are recorded ####


artificial_df = artificial_df[artificial_df['ReqDuration'] > 0]

# aggregated data frame: by 5min
artificial_df['TimeStampTrunc'] = artificial_df['TimeStamp'].values.astype('<M8[5m]')
agg_artificial_df = artificial_df.groupby(['TimeStampTrunc']).mean()

# create data frame of all timestamps of anomlies
h=['time stamps']
df_timestamps_with_outlier=DataFrame(timestamps_with_outlier,columns=h)
df_timestamps_with_outlier = df_timestamps_with_outlier.sort_values(by = 'time stamps')
df_timestamps_with_outlier.index = range(len(timestamps_with_outlier))


#artificial_df.plot(x='TimeStamp',y='ReqDuration')
#plt.show()
#agg_artificial_df.plot(y='ReqDuration')
#plt.show()


#df_recorded_outliers.to_csv('anomalies.csv')
#artificial_df.to_csv('synthetic_data.csv')
#df_timestamps_with_outlier.to_csv('anomlies_timestamps.csv')

